In [116]:
import ipywidgets
from ipywidgets import widgets, Layout

### Step 1

Loading the POS-Tagged Twitter dataset and seperating the word tags from the words

In [18]:
words_tags = []
with open('daily547.conll','r',encoding='utf-8') as f:
    for line in f:
        words_tags.append(tuple(line.split()))

In [67]:
words = []
tags = []
for tag in words_tags:
    if len(tag)<2:
        continue
    words.append(tag[0])
    tags.append(tag[1])

In [70]:
word_int = dict([(y,x) for x,y in enumerate(sorted(set(words)))])
tag_int = dict([(y,x) for x,y in enumerate(sorted(set(tags)))])

In [72]:
int_word = dict([(y,x) for x,y in word_int.items()])
int_tag = dict([(y,x) for x,y in tag_int.items()])

In [78]:
num_words = len(set(words))
num_tags = len(set(tags))
tag_count = len(words_tags)

In [77]:
trans_prob = [[0 for y in range(num_tags)] for x in range(num_tags)]
emis_prob = [[0 for y in range(num_words)] for x in range(num_tags)]

### Step 2

Calculating both transmission and emission probabilities

#### Transmission probs

In [80]:
i=0
while (i+1) < tag_count:
    if len(words_tags[i])<2 or len(words_tags[i+1])<2 or words_tags[i][0]=='.':
        i+=1
        continue
    prev_tag = words_tags[i][1]
    next_tag = words_tags[i+1][1]
    trans_prob[tag_int[prev_tag]][tag_int[next_tag]]+=1
    i+=1

In [82]:
for i in range(num_tags):
    tot_sum = sum(trans_prob[i])
    for j in range(num_tags):
        trans_prob[i][j]/=tot_sum

Emission probs

In [83]:
i=0
while i< tag_count:
    if len(words_tags[i])<2 :
        i+=1
        continue
    word = words_tags[i][0]
    tag = words_tags[i][1]
    emis_prob[tag_int[tag]][word_int[word]]+=1
    i+=1

In [84]:
for i in range(num_tags):
    tot_sum = sum(emis_prob[i])
    for j in range(num_words):
        emis_prob[i][j]/=tot_sum

### Step 3

Markov Process

In [104]:
import pandas as pd
import numpy as np
 
 
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
 
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            #                  ((1x2) . (1x2))      *     (1)
            #                        (1)            *     (1)
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
 
    return alpha
 
 
def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
    return beta
 
 
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)
 
    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
 
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return (a, b)
 
 
def viterbi(V, a, b, initial_distribution):
    global int_tag
    T = V.shape[0]
    M = a.shape[0]
 
    omega = np.zeros((T, M))
    omega[0, :] = np.log(initial_distribution * b[:, V[0]])
 
    prev = np.zeros((T - 1, M))
 
    for t in range(1, T):
        for j in range(M):
            # Same as Forward Probability
            probability = omega[t - 1] + np.log(a[:, j]) + np.log(b[j, V[t]])
 
            # This is our most probable state given previous state at time t (1)
            prev[t - 1, j] = np.argmax(probability)
 
            # This is the probability of the most probable state (2)
            omega[t, j] = np.max(probability)
 
    # Path Array
    S = np.zeros(T)
 
    # Find the most probable last hidden state
    last_state = np.argmax(omega[T - 1, :])
 
    S[0] = last_state
 
    backtrack_index = 1
    for i in range(T - 2, -1, -1):
        S[backtrack_index] = prev[i, int(last_state)]
        last_state = prev[i, int(last_state)]
        backtrack_index += 1
 
    # Flip the path array since we were backtracking
    S = np.flip(S, axis=0)
 
    # Convert numeric values to actual hidden states
    result = []
    for s in S:
        result.append(int_tag[s])
 
    return result
 
 


In [107]:
inp = 'I can talk'

In [108]:
val = np.array(list(map(lambda x: word_int[x],inp.split())))

In [109]:
a = np.array(trans_prob)

In [110]:
b = np.array(emis_prob)

In [111]:
init_dis = np.array([ 1/num_tags for i in range(num_tags)])

In [103]:
init_dis.shape

(25,)

In [114]:
a, b = baum_welch(val, a, b, init_dis, n_iter=100)

In [115]:
 print(viterbi(val, a, b, init_dis))

['!', '!', '!']
